In [ ]:
# Input information
email_dump    = "2020.06.04 182340 email summary.csv"

# SA2-139E
meeting_dates = ['2020-05-31 12:00','2020-06-05']

In [ ]:
# You will need pandas/numpy to run the first part of the script. Then plotly for the second part. You can install plotly via pip (pip install plotly)
# If the PNG export is not working, most probably orca is not installed. You can install it by executing in the Anaconda cmd: conda install -c plotly plotly-orca
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
df = pd.read_csv(
    email_dump,
    index_col=0)
df['date']    = pd.to_datetime(df['date'])
df['day']     = df['date'].dt.floor('d')
df['hour']    = df['date'].dt.floor('h')
df['company'] = df['sender address'].str.split('@', n = -1, expand = False).apply(lambda x: x[-1])

In [ ]:
df

In [ ]:
email_count_per_ai   = df.groupby('ai').count()
email_count_per_day  = df.groupby('day').count()
email_count_per_hour = df.groupby('hour').count()

In [ ]:
max_rows = pd.options.display.max_rows
pd.options.display.max_rows = None
emails_count = df.groupby('sender name').count().sort_values(by='date', ascending=False)
# See https://stackoverflow.com/questions/49574817/fill-a-new-pandas-column-with-row-numbers
emails_count['ranking'] = np.arange(len(emails_count)) + 1
emails_count['emails'] = emails_count['date']
display(emails_count[['ranking', 'emails']].iloc[0:25,:])
pd.options.display.max_rows = 60

In [ ]:
max_rows = pd.options.display.max_rows
pd.options.display.max_rows = None
emails_count_company = df.groupby('company').count().sort_values(by='date', ascending=False)

# See https://stackoverflow.com/questions/49574817/fill-a-new-pandas-column-with-row-numbers
emails_count_company['ranking'] = np.arange(len(emails_count_company)) + 1
emails_count_company['emails'] = emails_count_company['date']
display(emails_count_company[['ranking', 'emails']].iloc[0:20,:])
pd.options.display.max_rows = 60

In [ ]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [ ]:
data = go.Bar(
    x=email_count_per_ai.index, 
    y=email_count_per_ai['date'])
layout = go.Layout(
    xaxis= {'type': 'category'},
    title='Emails per #AI (total {0})'.format(email_count_per_ai['date'].sum()))
fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_html( "Emails per AI.html", auto_open=True)
fig.write_image("Emails per AI.png")

In [ ]:
# Data source for emails/day per AI
email_count_per_day_and_ai = df.groupby(['ai', 'day']).count()

ai_plot = {}
ais = df.ai.unique()
for ai in ais:
    ai_plot[ai] = email_count_per_day_and_ai[email_count_per_day_and_ai.index.isin([ai], level=0)]['date']

In [ ]:
layout = go.Layout(
    title='Emails per day and AI (total {0})'.format(email_count_per_ai['date'].sum()),
    xaxis_tickformat = '%d.%m %Hh',
    xaxis_range = meeting_dates)

fig = go.Figure(layout=layout)

for ai in ais:
    bar_data = ai_plot[ai]
    bar_y = bar_data
    bar_x = bar_data.index.get_level_values(1)
    bar_title = ai
    data = go.Bar(
        x=bar_x, 
        y=bar_y,
        name=bar_title)
    fig.add_trace(data)

fig.update_layout(barmode='relative')
fig.show()
fig.write_html( "Emails per day and AI.html", auto_open=True)
fig.write_image("Emails per day and AI.png")

In [ ]:
# Data source for emails/h per AI
email_count_per_hour_and_ai = df.groupby(['ai', 'hour']).count()

ai_plot = {}
ais = df.ai.unique()
for ai in ais:
    ai_plot[ai] = email_count_per_hour_and_ai[email_count_per_hour_and_ai.index.isin([ai], level=0)]['date']

In [ ]:
layout = go.Layout(
    title='Emails per hour and AI (total {0})'.format(email_count_per_ai['date'].sum()),
    xaxis_tickformat = '%d.%m %Hh',
    xaxis_range = meeting_dates)

fig = go.Figure(layout=layout)

for ai in ais:
    bar_data = ai_plot[ai]
    bar_y = bar_data
    bar_x = bar_data.index.get_level_values(1)
    bar_title = ai
    data = go.Bar(
        x=bar_x, 
        y=bar_y,
        name=bar_title)
    fig.add_trace(data)

fig.update_layout(barmode='relative')
fig.show()
fig.write_html( "Emails per hour and AI.html", auto_open=True)
fig.write_image("Emails per hour and AI.png")